In [75]:
import pandas as pd
import numpy as np
import base64
import requests
import datetime

In [76]:
data = pd.read_csv('test_book.csv')
data.head()

df = data[['userId', 'movieId', 'rating']]
df = df.iloc[0: 1000, :]
#df.loc[df['userId'] == 1,]

df

,userId,movieId,rating
0,1,2,1
1,1,3,2
2,1,4,2
3,1,5,5
4,1,7,4
5,1,8,3
6,1,9,5
7,2,1,1
8,2,2,5
9,2,3,3


In [77]:
df.groupby('movieId').userId.agg(list).reset_index()

,movieId,userId
0,1,"[2, 3, 4, 5]"
1,2,"[1, 2, 3, 4, 5]"
2,3,"[1, 2, 3, 4, 5]"
3,4,"[1, 4, 5]"
4,5,"[1, 2, 3, 5]"
5,6,"[2, 3, 4, 5]"
6,7,"[1, 2, 3, 4, 5]"
7,8,"[1, 2, 3, 4, 5]"
8,9,[1]


In [78]:
user_rated_movies_df = df.groupby('userId').movieId.agg(list).reset_index()
user_rated_movies_df



,userId,movieId
0,1,"[2, 3, 4, 5, 7, 8, 9]"
1,2,"[1, 2, 3, 5, 6, 7, 8]"
2,3,"[1, 2, 3, 5, 6, 7, 8]"
3,4,"[1, 2, 3, 4, 6, 7, 8]"
4,5,"[1, 2, 3, 4, 5, 6, 7, 8]"


In [79]:
users_that_rated_movie = df.groupby('movieId').userId.agg(list).reset_index().set_index('movieId').T.to_dict()
users_that_rated_movie

{1: {'userId': [2, 3, 4, 5]},
 2: {'userId': [1, 2, 3, 4, 5]},
 3: {'userId': [1, 2, 3, 4, 5]},
 4: {'userId': [1, 4, 5]},
 5: {'userId': [1, 2, 3, 5]},
 6: {'userId': [2, 3, 4, 5]},
 7: {'userId': [1, 2, 3, 4, 5]},
 8: {'userId': [1, 2, 3, 4, 5]},
 9: {'userId': [1]}}

In [80]:
# NEED TO CONVERT DATAFRAME INTO A MATRIX WHERE USERS ARE THE ROW INDICES AND MOVIES ARE COLUMNS WITH EACH CELL ENTRY
# BEING THE RATED FOR (USER, MOVIE)

newf = df.pivot(index='userId', columns='movieId')
newf.columns = newf.columns.droplevel(0)

 
matrix = newf.copy()

#print(list(newf.loc[:, 2]).count(1.0))

#print(matrix.loc[1,1])

newf.reset_index(inplace=True)



newf.fillna('NaN', inplace=True)

newf

movieId,userId,1,2,3,4,5,6,7,8,9
0,1,NaN,1.0,2.0,2.0,5.0,NaN,4.0,3.0,5.0
1,2,1.0,5.0,3.0,NaN,2.0,3.0,4.0,3.0,NaN
2,3,1.0,1.0,2.0,NaN,2.0,4.0,4.0,5.0,NaN
3,4,3.0,2.0,2.0,3.0,NaN,1.0,3.0,2.0,NaN
4,5,5.0,1.0,5.0,5.0,4.0,4.0,5.0,2.0,NaN


In [81]:
def get_alpha():
    return 0.01

In [82]:
print('Enter user id that wants a recommendation:')
test_userId = int(input())
print('Enter movie id for unrated movie: ')
unrated_movieId = int(input())

Enter user id that wants a recommendation:
Enter movie id for unrated movie: 


In [83]:
all_users = sorted(list(set(df['userId'])))


def compute_prior(list_of_movies, possible_ratings):
    # Let r_i be the rating of the i-th movie by any user
    # P(r_i = y) where y is any user:
    #           # users | r_u,i = y 
    #           -----------------------
    #           # users | r_u,i = NA + # of plausible ratings
    # i.e. P() a movie is rated y = # of usres that rated it y / # of users that rate =/= 'NA' + # of ratings for the movie
    prior_probs = {}
    alpha = get_alpha()

    # prior represents the prob that the item i be rated by ANY user as y

    for movie in list_of_movies:
        for rating in possible_ratings:
            num = len(df.loc[(df['movieId'] == movie) & (df['rating'] == rating) ]) + alpha
            denom = len(users_that_rated_movie[movie]['userId']) + len(possible_ratings) * alpha
            prior_probs[(movie, rating)] = num / denom
        
    
    return prior_probs

list_of_movies = list(set(df['movieId']))
possible_ratings = sorted(list(set(df['rating'])))
prior_probs = compute_prior(list_of_movies, possible_ratings)
prior_probs

{(1, 1): 0.4962962962962963,
 (1, 2): 0.0024691358024691358,
 (1, 3): 0.24938271604938272,
 (1, 4): 0.0024691358024691358,
 (1, 5): 0.24938271604938272,
 (2, 1): 0.596039603960396,
 (2, 2): 0.2,
 (2, 3): 0.0019801980198019802,
 (2, 4): 0.0019801980198019802,
 (2, 5): 0.2,
 (3, 1): 0.0019801980198019802,
 (3, 2): 0.596039603960396,
 (3, 3): 0.2,
 (3, 4): 0.0019801980198019802,
 (3, 5): 0.2,
 (4, 1): 0.0032786885245901644,
 (4, 2): 0.3311475409836066,
 (4, 3): 0.3311475409836066,
 (4, 4): 0.0032786885245901644,
 (4, 5): 0.3311475409836066,
 (5, 1): 0.0024691358024691358,
 (5, 2): 0.4962962962962963,
 (5, 3): 0.0024691358024691358,
 (5, 4): 0.24938271604938272,
 (5, 5): 0.24938271604938272,
 (6, 1): 0.24938271604938272,
 (6, 2): 0.0024691358024691358,
 (6, 3): 0.24938271604938272,
 (6, 4): 0.4962962962962963,
 (6, 5): 0.0024691358024691358,
 (7, 1): 0.0019801980198019802,
 (7, 2): 0.0019801980198019802,
 (7, 3): 0.2,
 (7, 4): 0.596039603960396,
 (7, 5): 0.2,
 (8, 1): 0.0019801980198019802

In [84]:
from numpy import NaN
from sympy import numer


def compute_likelihoood(list_of_movies, possible_ratings, list_of_users):
    # need to calculate likelihood
    # P(r_j = k | r_i = y) where j is the j-th movie being rated as 'k', given that the rating of the i-th movie was 'y'
    # ---------------> # users | r_u,j = k AND r_u,i = y
    #                 ----------------------------------------
    #               # users | r_u,j = NA AND r_u,i = y + # of possibilities
    #
    # ======== # of users that rated movie j as 'k' AND movie i rated as 'y' 
    #         ---------------------------------------------------------------
    #       # of users that rated movie j as NA and rated movie i as y + all possibilities
    movieId = 0
    rating = 1
    feature_probs = {}
    alpha = get_alpha()
    # posterior_prob = prior_prob P(r_i = y) * P(r_j = k | r_i = y) for every movie j rated by user u
    
    # ONLY THE LIKELIHOOD THAT RELATES TO THE TARGET ITEM WITH EACH ITEM RATED BY THE ACTED USER MUST BE COMPUTED.
    # as an example, let's use userId 1 for movieId 1 as this does not appear in dataset (NA)
    #test_movieId = 2
    
    #test_userId = 1
    #unrated_movieId = 1
    #print(test_userId, unrated_movieId)

    user_rated_movies = list(user_rated_movies_df.loc[user_rated_movies_df['userId'] == test_userId, ]['movieId'])[0]
    #print(user_rated_movies_df.loc[df['userId'] == test_userId])
    print(f"Movies that user {test_userId} rated {user_rated_movies}")

    # of users that rted moviedId 2 as 1 AND rated moviedId 1 as 1
    #numerator = len(newf.loc[(newf[test_movieId] == rating) & (newf[unrated_movieId] == rating)]) + alpha
    #denominator = len(newf.loc[(newf[test_movieId] != 'NaN') & (newf[unrated_movieId] == rating)]) + len(possible_ratings) * alpha
    #likelihood = numerator / denominator
   

    # likelihood values will only be calculated for movies that are rated by the specific user
    likelihoods = []
    
    likelihood_df = pd.DataFrame(columns=possible_ratings)

    #print(newf)
    for i in range(len(user_rated_movies)):
        rated_movieId = user_rated_movies[i]
        #likelihoods = []
        for possible_rating in possible_ratings:
          
            rating = matrix.loc[test_userId, rated_movieId]
            #print(rated_movieId, rating)
            numerator = len(newf.loc[(newf[rated_movieId] == rating) & (newf[unrated_movieId] == possible_rating)]) + alpha
            denominator = len(newf.loc[(newf[rated_movieId] != 'NaN') & (newf[unrated_movieId] == possible_rating)]) + len(possible_ratings) * alpha
            likelihood = numerator / denominator
            #print(f'rating: {rating} numerator: {numerator} denominator: {denominator} likelihood: {likelihood}')
            likelihoods.append(likelihood)
            """
            if len(likelihoods) == len(user_rated_movies):
                likelihood_df[possible_rating] = likelihoods"""
        likelihood_df.loc[rated_movieId] = likelihoods
        likelihoods = []
        
                
    #print(likelihoods)
    
    # Turn this into a dataframe where each possible rating is a column
    #print(likelihood_df)

    print(likelihood_df)

    return likelihood_df
    

    

list_of_users = list(set(df['userId']))

likelihood_df = compute_likelihoood(list_of_movies, possible_ratings, all_users)

Movies that user 2 rated [1, 2, 3, 5, 6, 7, 8]
     1    2    3    4         5
1  0.2  0.2  0.2  0.2  0.200000
2  0.2  0.2  0.2  0.2  0.009524
3  0.2  0.2  0.2  0.2  0.009524
5  0.2  0.2  0.2  0.2  0.009524
6  0.2  0.2  0.2  0.2  0.200000
7  0.2  0.2  0.2  0.2  0.961905
8  0.2  0.2  0.2  0.2  0.961905


In [85]:
def compute_posterior_probs(likelihood_df, prior_probs):
    # first compute product of each column (feature) in likelihood_df
    #unrated_movieId = 1

    li = likelihood_df[2]
    
    #print(likelihood_df.product(axis=0))

    col_product = pd.DataFrame(likelihood_df.product()).T

    #print(pd.DataFrame(likelihood_df.product()).T)

    # multiply each product by corresponding prior (class) prob to get posterior prob
    
    #print(col_product)
    posterior_probs = []
    for i in range(len(col_product.columns)):
        #print(prior_probs[(unrated_movieId, col_product.columns[i])], col_product.iloc[0, i])
        posterior_prob = prior_probs[(unrated_movieId, col_product.columns[i])] * col_product.iloc[0, i]
        #print(posterior_prob)
        posterior_probs.append(posterior_prob)

    # make the posterior probs into a dataframe
    #print(posterior_probs)
    posterior_probs_df = pd.DataFrame(posterior_probs).T
    posterior_probs_df.columns = col_product.columns

    print(posterior_probs_df)

    return posterior_probs_df

posterior_probs_df = compute_posterior_probs(likelihood_df=likelihood_df, prior_probs=prior_probs)

              1             2             3             4             5
0  1.219048e-07  1.219048e-07  1.219048e-07  1.219048e-07  3.075306e-08


In [86]:
def arg_max(posterior_probs_df):
    max_rating = np.argmax(posterior_probs_df) + 1
    #print(np.argmax(posterior_probs_df))
    return max_rating

rating = arg_max(posterior_probs_df)

def recommended(rating):
    recommended = ""
    if rating < 3:
        recommended = "User will most likely not enjoy this"
    elif rating == 3:
        recommended = "User will most likely find this to be OK"
    elif rating > 3 and rating < 5:
        recommended = "User will most likely enjoy this"
    elif rating == 5:
        recommended = "User should definitely watch this"

    return recommended

recommended = recommended(rating)

print(f'The most likely rating for movie {unrated_movieId} based on users similar to user {test_userId} is a rating of {rating}. {recommended}')

The most likely rating for movie 9 based on users similar to user 2 is a rating of 1. User will most likely not enjoy this


In [87]:
'https://www.youtube.com/watch?v=3I8oX3OUL6I'
'https://www.youtube.com/watch?v=lFJbZ6LVxN8'
'https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8787761'

'https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8787761'